## Import packages

In [1]:
cd /Users/zoltan/Dropbox/Python

/Users/zoltan/Dropbox/Python


In [2]:
pwd

u'/Users/zoltan/Dropbox/Python'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import cline_analysis as ca
import pandas as pd
import seaborn as sns
import datetime
import os
from scipy.signal import medfilt
import functools
from scipy.optimize import bisect
from scipy import stats

sns.set_style("whitegrid")
sns.set_style("ticks")
%matplotlib qt
%config InlineBackend.figure_format = 'svg'
plt.matplotlib.rcParams['svg.fonttype'] = 'svgfont' # fonts will be recognized by Adobe Illustrator

## Load data

In [4]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Tarauaca/csv_files/'
fnames,clxs,clys,rbxs,lbxs,rbys,lbys,curvatures,ages,widths,dates = ca.load_data(dirname)

In [5]:
fnames

['Tarauaca_20070729.csv',
 'Tarauaca_20170724.csv',
 'Tarauaca_Jurua_20070729.csv',
 'Tarauaca_Jurua_20170724.csv']

In [6]:
dates

[datetime.datetime(2007, 7, 29, 0, 0),
 datetime.datetime(2017, 7, 24, 0, 0),
 datetime.datetime(2007, 7, 29, 0, 0),
 datetime.datetime(2017, 7, 24, 0, 0)]

In [7]:
plt.figure()
plt.plot(rbxs[2],rbys[2])
plt.axis('equal')

(205809.21886044997,
 430003.05707055004,
 -780816.9767638999,
 -722784.43243809999)

## Get migration rate

In [8]:
ts1 = 2 # first timestep
ts2 = 3 # second timestep

d = dates[ts2]-dates[ts1]
years = d.days/365.0

x = np.array(clxs[ts1])
y = np.array(clys[ts1])

x = x[:22262] # get rid of extra length in x and y
y = y[:22262]

xn = np.array(clxs[ts2])
yn = np.array(clys[ts2])

migr_rate, migr_sign, p, q = ca.get_migr_rate(x,y,xn,yn,years,0)

In [196]:
migr_rate = medfilt(savgol_filter(migr_rate,41,3),kernel_size=5) # smoothing
curv,s = ca.compute_curvature(x,y)
curv = medfilt(savgol_filter(curv,71,3),kernel_size=5) # smoothing

In [197]:
# set intervals affected by cu=toffs to NaN - specific to Jurua (Tarauaca) river segment
migr_rate[10408:10712] = np.NaN
migr_rate[14725:15051] = np.NaN
migr_rate[19650:20111] = np.NaN

In [198]:
plt.figure()
plt.plot(migr_rate)

In [9]:
from shapely.geometry import LineString
import geopandas as gpd

def write_shapefile(x,y,epsg,dirname,filename):
    coords=[]
    for i in range(len(x)):
        coords.append((x[i],y[i]))
    ls = LineString(coords)
    ls = gpd.GeoSeries(ls)
    ls.crs = {'init' :'epsg:'+str(epsg)}
    ls.to_file(dirname+filename+'.shp')
    
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Curvature_paper_II/GIS_data/Jurua_Tarauaca/'
epsg = 32619

year = str(dates[ts1].year)
day = str(dates[ts1].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts1].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(x,y,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts1],lbys[ts1],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts1],rbys[ts1],epsg=epsg,dirname=dirname,filename=filename)

year = str(dates[ts2].year)
day = str(dates[ts2].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts2].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(xn,yn,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts2],lbys[ts2],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts2],rbys[ts2],epsg=epsg,dirname=dirname,filename=filename)

## Read 'valid' inflection points and corresponding points of zero migration from CSV file

In [199]:
df = pd.read_csv('Tarauaca_Jurua_LT05_L1TP_004065_20070729_20161112_01_T1_inflection_and_zero_migration_indices.csv')
LZC = np.array(df['index of inflection point'])
LZM = np.array(df['index of zero migration'])

In [200]:
# indices of bends affected by low erodibility and cutoffs (these have been picked manually)
erodibility_inds = [1,5,11,91,95,97,101,105,267]
cutoff_inds = [10,38,39,61,62,67,210,211,270,271]

# indices of boundaries between river segments
ind1 = 9183
ind2 = 126
ind3 = 21736
ind4 = 276

plt.figure()
plt.plot(widths[2])
plt.plot([ind1,ind1],[0,300],'r')
plt.plot([ind3,ind3],[0,300],'r')

## Plot curvature and migration rate series side-by-side

In [201]:
# plot curvature and migration rate along the channel

fig, ax1 = plt.subplots(figsize=(25,4))
plt.tight_layout()
    
curv_scale = 0.4
migr_scale = 6
y1 = curv_scale
y2 = -3*curv_scale
y3 = 3*migr_scale
y4 = -migr_scale

y5 = -2*curv_scale
y6 = 2*migr_scale

for i in range(0,len(LZC)-1,2):
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,0.85,0.85],edgecolor='k',zorder=0)

deltas = 25.0
ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_rate, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y2,y1)
ax2.set_ylim(y4,y3)
ax1.set_xlim(0,s[-1])

for i in erodibility_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[1.0,0.85,0.85],zorder=0) 
    
for i in cutoff_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,1.0,0.85],zorder=0) 
    
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y1,y1,0,y5,y2,y2,y5,0]
        ax1.fill(xcoords,ycoords,color='w') 
        
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y3,y3,y6,0,y4,y4,0,y6]
        ax2.fill(xcoords,ycoords,color='w') 

for i in range(0,len(LZC)-1,2):
    ax1.text(s[LZC[i]],0.4,str(i),fontsize=12)

## Estimate lag between curvature and migration rate

In [202]:
np.mean(widths[2][:ind1]) # mean width of first segment

119.59575669613805

In [203]:
np.mean(widths[2][ind1:ind3]) # mean width of second segment

148.99501801859674

In [204]:
window_length = 500
time_shifts = ca.get_time_shifts(migr_rate[:ind1],curv[:ind1],window_length)

plt.figure()
plt.plot(time_shifts)

# average lag for first segment
25.0*np.round(np.mean(time_shifts))

-425.0

In [205]:
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
# average lag for first segment
np.mean(25.0*(LZM[:ind2]-LZC[:ind2]))

441.26984126984127

In [206]:
window_length = 500
time_shifts = ca.get_time_shifts(migr_rate[ind1:ind3],curv[ind1:ind3],window_length)

plt.figure()
plt.plot(time_shifts)

# average lag for second segment
25.0*np.round(np.mean(time_shifts))

-550.0

In [207]:
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
# average lag for second segment
np.mean(25.0*(LZM[ind2:ind4]-LZC[ind2:ind4]))

569.5

# First segment (Jurua T A)

## Estimate friction factor Cf

In [208]:
# first segment
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 0
i2 = ind1
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [209]:
# this might take a while to run
kl = 20.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[2][:ind1])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.00604765625


In [210]:
Cf_opt = 0.00604765625

In [211]:
W

119.59575669613805

## Estimate migration rate constant kl

In [212]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
for i in np.arange(5,30):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(5,30),errors);

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [213]:
kl_opt = 12.0 # the error is at minimum for kl = 12.0

## Plot actual migration rate against nominal migration rate

In [214]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.mean(widths[2][:ind1]) 
curv_nodim = w*curv_t*kl_opt
lag = 18
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 7.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--')
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [215]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.762657644408
0.581646682574
0.0


In [216]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

9165

In [217]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [218]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [219]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.75226488193
0.565902452585
0.0


In [220]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

5.4273042152070659

In [221]:
max_m = 7.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [223]:
max_m = 10.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    if i<ind2:
        plt.scatter(R1[LZC[i]:LZC[i+1]][::5],migr_rate_t[LZC[i]:LZC[i+1]][::5],c='r',s=15)
for i in cutoff_inds:
    if i<ind2:
        plt.scatter(R1[LZC[i]:LZC[i+1]][::5],migr_rate_t[LZC[i]:LZC[i+1]][::5],c='g',s=15)

# Second segment (Jurua T B)

## Estimate friction factor Cf

In [224]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = ind1
i2 = ind3
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [225]:
# eliminate cutoffs
migr_rate_t[-i1+10408:-i1+10712] = np.NaN
migr_rate_t[-i1+14725:-i1+15051] = np.NaN
migr_rate_t[-i1+19650:-i1+20111] = np.NaN
plt.figure()
plt.plot(migr_rate_t);

In [226]:
# this might take a while to run
kl = 20.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[2][ind1:ind3])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

# eliminate NaNs:
curv_t = curv_t[np.isnan(migr_rate_t)==0]
s = s[np.isnan(migr_rate_t)==0]
migr_rate_t = migr_rate_t[np.isnan(migr_rate_t)==0]

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.004878125


In [227]:
Cf_opt = 0.004878125

In [228]:
W

148.99501801859674

## Estimate migration rate constant kl

In [229]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
for i in np.arange(5,30):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t),75)))
    
plt.figure()
plt.plot(np.arange(5,30),errors);

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [230]:
kl_opt = 11.0 # the error is at minimum for kl = 11.0

In [231]:
569/25.0

22.76

## Plot actual migration rate against nominal migration rate

In [232]:
# need to recalculate migration rate 
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = ind1
i2 = ind3
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

# eliminate cutoffs
migr_rate_t[-i1+10408:-i1+10712] = np.NaN
migr_rate_t[-i1+14725:-i1+15051] = np.NaN
migr_rate_t[-i1+19650:-i1+20111] = np.NaN

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5) # smoothing

# kernel density and scatterplot of actual vs. nominal migration rate
w = np.mean(widths[2][ind1:ind3]) 
curv_nodim = w*curv_t*kl_opt
lag = 23
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 7.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--')
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [233]:
plt.figure()
plt.plot(migr_rate_t)
plt.plot(kl_opt*w*curv_t)

In [234]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.781202091894
0.61027670838
0.0


In [235]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

11319

In [236]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [237]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r');

In [239]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.768725607061
0.590939058952
0.0


In [240]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

4.639095189476051

In [241]:
max_m = 7.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [259]:
max_m = 10.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    if i>=ind2:
        plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::2],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::2],c='r',s=15)
for i in cutoff_inds:
    if i>=ind2:
        plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::2],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::2],c='g',s=15)